In [1]:
# --- Baseline packages ---
import pandas as pd
pd.set_option('display.max_columns', None)

# --- Utility packages ---
import os
import sys

# --- Helper functions ---
sys.path.append(os.path.abspath('../src'))
import ecg_utils

In [2]:
# Set global seeds.
ecg_utils.set_global_seeds(seed_num=42)

# Get runtime.
device = ecg_utils.check_runtime_device()

# Identify relevant folder.
final_results_dir = os.path.join("..", "final-results")

# Get eval metrics and model info.
metrics_df = ecg_utils.get_eval_metrics(final_results_dir)
model_info, model_stats = ecg_utils.get_model_info(final_results_dir, device)

Global seed for torch set.
Global seed for numpy and python's inbuilt RNG set.
Runtime type: mps
Training mode: fine tuning. All layers are trainable.
Successfully instantiated model with saved params from: ../final-results/convnextbase_config1_adj/best_model_retrained.pth
Model configuration complete for convnextbase

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.normalization.LayerNorm'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
Training mode: fine tuning. All layers are trainable.
Successfully instantiated model with saved params from: ../final-results/convnextbase_config2_adj/best_model_retrained.pth
Model configuration complete for convnextbase

[INFO] Register count_convNd() for

In [3]:
analysis_df = metrics_df.join(model_info.set_index('model_arch'), on='model_arch')

arch_dict = {
    'resnet18': ['resnet', 'small'], 'resnet50': ['resnet', 'big'],
    'efficientnetb0': ['efficientnet', 'small'], 'efficientnetb3': ['efficientnet', 'big'],
    'convnexttiny': ['convnext', 'small'], 'convnextbase': ['convnext', 'big'],
    'mobilenetv3small': ['mobilenetv3', 'small'], 'mobilenetv3large': ['mobilenetv3', 'big']
}
analysis_df['arch_family'] = analysis_df['model_arch'].map({key: value[0] for  key, value in arch_dict.items()})
analysis_df['arch_variant'] = analysis_df['model_arch'].map({key: value[1] for  key, value in arch_dict.items()})

analysis_df.head()

,model_arch,dataset_config,dataset_type,metric_name,label_name,results,eval_env,eval_gpu,total_params,mac_gb,input_size_mb,pass_size_mb,param_size_mb,total_size_mb,mac_gb_wrong,arch_family,arch_variant
0,convnextbase,config1,train,accuracy,AFIB,0.988726,slurm,cuda,87570564,15.372426,0.602184,276.176928,350.210064,626.989176,0.64551,convnext,big
1,convnextbase,config1,train,accuracy,GSVT,0.990044,slurm,cuda,87570564,15.372426,0.602184,276.176928,350.210064,626.989176,0.64551,convnext,big
2,convnextbase,config1,train,accuracy,SB,0.998714,slurm,cuda,87570564,15.372426,0.602184,276.176928,350.210064,626.989176,0.64551,convnext,big
3,convnextbase,config1,train,accuracy,SR,0.994938,slurm,cuda,87570564,15.372426,0.602184,276.176928,350.210064,626.989176,0.64551,convnext,big
4,convnextbase,config1,train,accuracy,macro_avg,0.993979,slurm,cuda,87570564,15.372426,0.602184,276.176928,350.210064,626.989176,0.64551,convnext,big


In [4]:
output_filepath = os.path.join(final_results_dir, "analysis_df.csv")
analysis_df.to_csv(output_filepath, index=False)